In [1]:
import numpy as np 
import pandas as pd 
import pymongo
from pymongo import MongoClient
#Importando librerias

In [2]:
#Conectando con Mogo
client = MongoClient('mongodb://localhost:27017')

In [3]:
#llamando companies
db=client.companies


In [4]:
#Guardando en curr con querys de filtrado de datos
curr_ekchuah = db.companies.find ({'$and': [
                                    {'category_code': 'ecommerce'},\
                                    {'offices': {'$not': {'$size':0}}},\
                                    {'founded_year': {'$lt':2012}}
                                    ]},\
                         {'_id':0,'name':1, 'offices':1, 'founded_year':1, 'products':1}).sort('ipo.valuation_amount',1)


In [5]:
#convirtiendo en df
df=pd.DataFrame(curr_ekchuah)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          464 non-null    object
 1   founded_year  464 non-null    int64 
 2   products      464 non-null    object
 3   offices       464 non-null    object
dtypes: int64(1), object(3)
memory usage: 14.6+ KB


In [6]:
df

,name,founded_year,products,offices
0,Wize,2006,[],"[{'description': '', 'address1': '1110 Burling..."
1,Zlio,2005,"[{'name': 'Zlio', 'permalink': 'zlio'}]","[{'description': '', 'address1': '55 rue Serva..."
2,TheFind,2006,"[{'name': 'TheFind.com', 'permalink': 'thefind...","[{'description': '', 'address1': '310 Villa St..."
3,Zazzle,1999,"[{'name': 'Zazzle', 'permalink': 'zazzle-com'}]","[{'description': '', 'address1': '1800 Seaport..."
4,Kaboodle,2005,"[{'name': 'Kaboodle', 'permalink': 'kaboodle'}]","[{'description': '', 'address1': '640 W. Calif..."
...,...,...,...,...
459,Lift Media,2008,[],"[{'description': 'HQ', 'address1': '332 Pine S..."
460,Nokaut,2006,[],"[{'description': 'Grupa Nokaut HQ', 'address1'..."
461,Shutterfly,1999,[],"[{'description': '', 'address1': '2800 Bridge ..."
462,Higher One,2000,[],"[{'description': 'HQ', 'address1': '25 Science..."


In [7]:

new_df = pd.DataFrame(index=[], columns=df.columns)
for _, i in df.iterrows():
    flattened_d = [dict(i.to_dict(), offices=c) for c in i.offices]
    new_df = new_df.append(flattened_d )

In [8]:
new_df

,name,founded_year,products,offices
0,Wize,2006,[],"{'description': '', 'address1': '1110 Burlinga..."
0,Zlio,2005,"[{'name': 'Zlio', 'permalink': 'zlio'}]","{'description': '', 'address1': '55 rue Servan..."
0,TheFind,2006,"[{'name': 'TheFind.com', 'permalink': 'thefind...","{'description': '', 'address1': '310 Villa Str..."
0,Zazzle,1999,"[{'name': 'Zazzle', 'permalink': 'zazzle-com'}]","{'description': '', 'address1': '1800 Seaport'..."
0,Kaboodle,2005,"[{'name': 'Kaboodle', 'permalink': 'kaboodle'}]","{'description': '', 'address1': '640 W. Califo..."
...,...,...,...,...
0,Lift Media,2008,[],"{'description': 'HQ', 'address1': '332 Pine St..."
0,Nokaut,2006,[],"{'description': 'Grupa Nokaut HQ', 'address1':..."
0,Shutterfly,1999,[],"{'description': '', 'address1': '2800 Bridge P..."
0,Higher One,2000,[],"{'description': 'HQ', 'address1': '25 Science ..."


In [9]:
#Preparando las colunmas que tienen datos anidados, offices
new_df1 = pd.DataFrame(index=[], columns=new_df.columns)
for _, i in new_df.iterrows():
    flattened_d = [dict(i.to_dict(), products=c) for c in i.products]
    new_df1 = new_df1.append(flattened_d )


In [10]:
new_df1 = new_df1.reset_index()

In [11]:
new_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         472 non-null    int64 
 1   name          472 non-null    object
 2   founded_year  472 non-null    object
 3   products      472 non-null    object
 4   offices       472 non-null    object
dtypes: int64(1), object(4)
memory usage: 18.6+ KB


In [12]:
from pandas.io.json import json_normalize
#Desanidando offices con json_normalize

new_df2=json_normalize(new_df1['offices'])

/home/sromero/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


In [13]:
new_df2
#En este df ya tenemos la información desanidada de offices

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,,55 rue Servan,,75011,Paris,None,FRA,48.862825,2.381836
1,,310 Villa Street,,94041,Mountain View,CA,USA,37.391502,-122.073463
2,,1800 Seaport,,94063,Redwood City,CA,USA,37.510854,-122.201356
3,,640 W. California Ave.,Suite 220,94041,Sunnyvale,CA,USA,37.382162,-122.036301
4,Vast San Francisco,560 Sutter St.,Suite 300,94108,San Francisco,CA,USA,37.787183,-122.397759
...,...,...,...,...,...,...,...,...,...
467,None,1200 12th Ave,S # 1200,98144,Seattle,WA,USA,47.592300,-122.317295
468,None,1200 12th Ave,S # 1200,98144,Seattle,WA,USA,47.592300,-122.317295
469,None,1200 12th Ave,S # 1200,98144,Seattle,WA,USA,47.592300,-122.317295
470,None,1200 12th Ave,S # 1200,98144,Seattle,WA,USA,47.592300,-122.317295


In [14]:
new_df2 = new_df2.drop(columns='description')


In [15]:
#Juntando los df
dfmap = pd.concat([new_df1, new_df2], axis=1, sort=False)

In [16]:
dfmap

,index,name,founded_year,products,offices,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,0,Zlio,2005,"{'name': 'Zlio', 'permalink': 'zlio'}","{'description': '', 'address1': '55 rue Servan...",55 rue Servan,,75011,Paris,None,FRA,48.862825,2.381836
1,0,TheFind,2006,"{'name': 'TheFind.com', 'permalink': 'thefind-...","{'description': '', 'address1': '310 Villa Str...",310 Villa Street,,94041,Mountain View,CA,USA,37.391502,-122.073463
2,0,Zazzle,1999,"{'name': 'Zazzle', 'permalink': 'zazzle-com'}","{'description': '', 'address1': '1800 Seaport'...",1800 Seaport,,94063,Redwood City,CA,USA,37.510854,-122.201356
3,0,Kaboodle,2005,"{'name': 'Kaboodle', 'permalink': 'kaboodle'}","{'description': '', 'address1': '640 W. Califo...",640 W. California Ave.,Suite 220,94041,Sunnyvale,CA,USA,37.382162,-122.036301
4,0,Vast,2001,"{'name': 'Vast', 'permalink': 'vast'}","{'description': 'Vast San Francisco', 'address...",560 Sutter St.,Suite 300,94108,San Francisco,CA,USA,37.787183,-122.397759
...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,36,Amazon,1994,"{'name': 'Exchange.com', 'permalink': 'exchang...","{'description': None, 'address1': '1200 12th A...",1200 12th Ave,S # 1200,98144,Seattle,WA,USA,47.592300,-122.317295
468,37,Amazon,1994,"{'name': 'Bookpages', 'permalink': 'bookpages'}","{'description': None, 'address1': '1200 12th A...",1200 12th Ave,S # 1200,98144,Seattle,WA,USA,47.592300,-122.317295
469,38,Amazon,1994,"{'name': 'Amazon Route 53', 'permalink': 'amaz...","{'description': None, 'address1': '1200 12th A...",1200 12th Ave,S # 1200,98144,Seattle,WA,USA,47.592300,-122.317295
470,39,Amazon,1994,"{'name': 'Kindle Fire', 'permalink': 'kindle-f...","{'description': None, 'address1': '1200 12th A...",1200 12th Ave,S # 1200,98144,Seattle,WA,USA,47.592300,-122.317295


In [17]:
dfmap = dfmap.drop(columns='offices')

In [18]:
dfmap = dfmap.drop(columns='index')

In [19]:
dfmap = dfmap.drop(columns='state_code')

In [20]:
dfmap

,name,founded_year,products,address1,address2,zip_code,city,country_code,latitude,longitude
0,Zlio,2005,"{'name': 'Zlio', 'permalink': 'zlio'}",55 rue Servan,,75011,Paris,FRA,48.862825,2.381836
1,TheFind,2006,"{'name': 'TheFind.com', 'permalink': 'thefind-...",310 Villa Street,,94041,Mountain View,USA,37.391502,-122.073463
2,Zazzle,1999,"{'name': 'Zazzle', 'permalink': 'zazzle-com'}",1800 Seaport,,94063,Redwood City,USA,37.510854,-122.201356
3,Kaboodle,2005,"{'name': 'Kaboodle', 'permalink': 'kaboodle'}",640 W. California Ave.,Suite 220,94041,Sunnyvale,USA,37.382162,-122.036301
4,Vast,2001,"{'name': 'Vast', 'permalink': 'vast'}",560 Sutter St.,Suite 300,94108,San Francisco,USA,37.787183,-122.397759
...,...,...,...,...,...,...,...,...,...,...
467,Amazon,1994,"{'name': 'Exchange.com', 'permalink': 'exchang...",1200 12th Ave,S # 1200,98144,Seattle,USA,47.592300,-122.317295
468,Amazon,1994,"{'name': 'Bookpages', 'permalink': 'bookpages'}",1200 12th Ave,S # 1200,98144,Seattle,USA,47.592300,-122.317295
469,Amazon,1994,"{'name': 'Amazon Route 53', 'permalink': 'amaz...",1200 12th Ave,S # 1200,98144,Seattle,USA,47.592300,-122.317295
470,Amazon,1994,"{'name': 'Kindle Fire', 'permalink': 'kindle-f...",1200 12th Ave,S # 1200,98144,Seattle,USA,47.592300,-122.317295


In [21]:
dfmap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          472 non-null    object 
 1   founded_year  472 non-null    object 
 2   products      472 non-null    object 
 3   address1      467 non-null    object 
 4   address2      463 non-null    object 
 5   zip_code      468 non-null    object 
 6   city          471 non-null    object 
 7   country_code  472 non-null    object 
 8   latitude      314 non-null    float64
 9   longitude     314 non-null    float64
dtypes: float64(2), object(8)
memory usage: 37.0+ KB


In [22]:
#Rellenando datos nulos con Unknown
dfmap[['address1', 'address2', 'zip_code']] = dfmap[['address1', 'address2', 'zip_code']].fillna('Unknwon')

In [23]:
#Rellenando datos nulos con 0.0
dfmap[['longitude', 'latitude']] = dfmap[['longitude', 'latitude']].fillna(0.0)

In [24]:
dfmap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          472 non-null    object 
 1   founded_year  472 non-null    object 
 2   products      472 non-null    object 
 3   address1      472 non-null    object 
 4   address2      472 non-null    object 
 5   zip_code      472 non-null    object 
 6   city          471 non-null    object 
 7   country_code  472 non-null    object 
 8   latitude      472 non-null    float64
 9   longitude     472 non-null    float64
dtypes: float64(2), object(8)
memory usage: 37.0+ KB


In [25]:
#Creando colección en mongodb
db.datos_mapa.insert_many(dfmap.to_dict('records'))

In [26]:
#Agregando 2dsphere
db.datos_mapa.create_index ([('principal', '2dsphere')])

'principal_2dsphere'

In [27]:
dfmap.latitude

0      48.862825
1      37.391502
2      37.510854
3      37.382162
4      37.787183
         ...    
467    47.592300
468    47.592300
469    47.592300
470    47.592300
471    47.592300
Name: latitude, Length: 472, dtype: float64

In [28]:
#Obteniendo datos que se usarán en mapas
dire = dfmap[['latitude','longitude']]
dire1= dire.values.tolist()

In [29]:
import folium 
from folium import plugins


In [30]:
#Primer mapa
#Realicé este mapa para con la finalidad de visualidar los lugares con presencia
#de e-commerce y observamos mayor presencia en USA y una parte de Europa.
mapa_ek= folium.Map([38.500000, -98.000000], zoom_start=3)
data = dfmap[['latitude', 'longitude']].values
mapa_ek.add_child(plugins.HeatMap(data, radius=15))


In [31]:
mapa_ek.save('mapa1.html')

In [32]:
#Se realizó este segundo mapa para identificar ubicaciones de e-commerce

map_ek2 = folium.Map(
    location=[38.500000, -98.000000],
     tiles='Stamen Terrain',
    zoom_start=4)
for point in range(0,len(dire1)):
    folium.Marker(dire1[point], popup=dfmap['name'][point]).add_to(map_ek2)
map_ek2    

In [33]:
map_ek2.save('mapa2.html')

In [34]:
#Se hizo este mapa para visualizar las ubicaciones de e-commerce en London
map_ek3 = folium.Map(
    location=[51.5024412,-0.0189649],
    zoom_start=7)
for point in range(0,len(dire1)):
    folium.Marker(dire1[point], popup=dfmap['name'][point]).add_to(map_ek3)
  
map_ek3

In [35]:
map_ek3.save('mapa3.html')

In [36]:
#Se estabelció una zona determinada como candidata para la ubicación del co-work
map_ek4 = folium.Map(
    location=[51.509865,-0.118092],
    zoom_start=10)

folium.CircleMarker(
        location=[51.509865,-0.118092],
        radius=70,
        popup='Laurelhurst Park',
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(map_ek4)
icon=folium.Icon(color='white', icon='car', icon_color="blue", prefix='fa')
for point in range(0,len(dire1)):
    folium.Marker(dire1[point], popup=dfmap['name'][point]).add_to(map_ek4)


map_ek4 

In [37]:
map_ek4.save('mapa4.html')

In [38]:
#El circulo rojo indica la ubicación del cowork  
map_ek5 = folium.Map(
    location=[51.509865,-0.118092],
    zoom_start=10)

folium.CircleMarker(
        location=[51.509865,-0.118092],
        radius=70,
        popup='Laurelhurst Park',
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(map_ek5)

folium.Circle(
    location=[51.5024412,-0.0189649],
    radius=200,
    popup='The Waterfront',
    color='crimson',
    fill=False,
).add_to(map_ek5)

folium.Marker([51.503827, 0.049267], popup= dfmap['name'][point],
               icon=folium.Icon(color='red',icon='plane')).add_to(map_ek5)
map_ek5

In [39]:
map_ek5.save('mapa5.html')